In [90]:
import pickle
ss = {"sp" : [1, 2, 3], "f0" : [4,5,6] }
dd = { "v" : [ss]*5}
# d = { "val" : dd, "key" : ss }
print(dd['v'][0])
# afile = open(r'd.pkl', 'wb')

# pickle.dump(d, afile)
# afile.close()

{'sp': [1, 2, 3], 'f0': [4, 5, 6]}


In [19]:
print(d['val']['sp'])
foo = pickle.load(open("d.pkl", "rb"))
print(foo['val']['sp'])


[1, 2, 3]
[1, 2, 3]


In [85]:
import os
import soundfile as sf
import pyworld as pw
import matplotlib.pyplot as plt

aiueo = ['a','i','u','e','o']
frame_period = 10.0 # millisecond
for x in os.walk('data'):

    for who in x[1]:
        for v in aiueo:
            path = os.path.join('data',who,v)
            print(path)
            x,fs = sf.read(path+'.wav')
            x = (x[:,0] + x[:,1])/2 
            x = x/max(x)*0.5
            f0, sp, _ = pw.wav2world(x, fs, frame_period =  frame_period)
            f0 = f0[1:-1]
            sp = sp[1:-1,:]
            file_cnt = 2
            while os.path.isfile(path+str(file_cnt)+'.wav'):
                print(path+str(file_cnt)+'.wav')
                x2,_ = sf.read(path+str(file_cnt)+'.wav')
                x2 = (x2[:,0] + x2[:,1])/2 # stereo to mono
                f02, sp2,_ = pw.wav2world(x2, fs, frame_period =  frame_period)    # use default options
                
                f0_note =np.concatenate((f0_note,f02[1:-1]), axis=0)
                sp =np.concatenate((sp,sp2), axis=0)
                
                file_cnt +=1
#             print(f0.shape,sp.shape)
            valid_idx = np.where(f0!=0)
            f0 =  f0[valid_idx]
            sp =  sp[valid_idx,:]
            
            f0_note = 12*np.log2(f0/440)+49
#             plt.plot(f0)
#             plt.show()
            
            print(f0.shape,sp.shape)
    #         sorted_note = np.array(sorted(i for i in f0_note if i < note_upper+1))
    #         del_note = abs(sorted_note[1:] - sorted_note[:-1])
       

data\LISA\a
(336,) (336, 1025)
(336,) (1, 336, 1025)
data\LISA\i
(558,) (558, 1025)
(558,) (1, 558, 1025)
data\LISA\u
(545,) (545, 1025)
(545,) (1, 545, 1025)
data\LISA\e
(418,) (418, 1025)
(418,) (1, 418, 1025)
data\LISA\o
(447,) (447, 1025)
(447,) (1, 447, 1025)
data\士峰\a
(1064,) (1064, 1025)
(1064,) (1, 1064, 1025)
data\士峰\i
(848,) (848, 1025)
(848,) (1, 848, 1025)
data\士峰\u
(628,) (628, 1025)
(628,) (1, 628, 1025)
data\士峰\e
(772,) (772, 1025)
(772,) (1, 772, 1025)
data\士峰\o
(884,) (884, 1025)
(884,) (1, 884, 1025)
data\子齊\a
(355,) (355, 1025)
(355,) (1, 355, 1025)
data\子齊\i
(291,) (291, 1025)
(291,) (1, 291, 1025)
data\子齊\u
(308,) (308, 1025)
(308,) (1, 308, 1025)
data\子齊\e
(358,) (358, 1025)
(358,) (1, 358, 1025)
data\子齊\o
(291,) (291, 1025)
(291,) (1, 291, 1025)
data\我\a
(317,) (317, 1025)
(317,) (1, 317, 1025)
data\我\i
data\我\i2.wav
(314,) (822, 1025)
(314,) (1, 314, 1025)
data\我\u
(369,) (369, 1025)
(369,) (1, 369, 1025)
data\我\e
(440,) (440, 1025)
(440,) (1, 440, 1025)
data\我\

In [ ]:
from termcolor import colored
import sys
import numpy as np
import os.path
import soundfile as sf
# import librosa
import pyworld as pw

#28 38 9 男
#28 38 9 女

frame_period = 10.0 # millisecond

aiueo = ['a','i','u','e','o']
for i in range(len(aiueo)):
    wavname = 'data/'+aiueo[i]+'.wav'
    if not os.path.isfile(wavname):
        print(colored('錯誤:找不到音檔','red'),wavname)
        continue

    x, fs = sf.read(wavname)
    x = (x[:,0] + x[:,1])/2 # stereo to mono

    num_of_frames = int(np.floor(len(x)/samples_per_frame))    
    f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)    # use default options


#   pitch judgement
    f0_note = 12*np.log2(f0[1:-1]/440)+49
    min_his = min(np.histogram(f0_note, bins=13)[0])

In [101]:
x, fs = sf.read('a.wav')

try:
    x = (x[:,0] + x[:,1])/2
    x = x/max(x)*0.3
except:
    pass
frame_period = 10
f0, sp, ap = pw.wav2world(x, fs, frame_period =  frame_period)    # use default options
f0 = f0[1:]
sp = sp[1:]
ap = ap[1:]

In [103]:
#return f0 index array with len (max_note-min_note)/note_step+1

def get_f0_index_tbl(f0,min_note,max_note,note_step=0.25):
    f0_note = 12*np.log2(f0/440)+49
    out_len = int((max_note-min_note)/note_step+1)
    note_lin =np.linspace(min_note, max_note, num=out_len)
    idx_out = np.zeros(out_len)
    for i in range(out_len):
        idx_out[i] = np.argmin(abs(f0_note-note_lin[i]))
    return idx_out
        

tbl = get_f0_index_tbl(f0,28,38)
    
        
    

In [104]:
print(tbl)

[ 40.  44.  46.  50.  52.  55.  57.  65.  70.  73.  77.  86.  95.  98.
 102. 113. 117. 121. 125. 129. 135. 144. 149. 158. 165. 175. 183. 192.
 196. 201. 215. 222. 233. 238. 245. 253. 257. 262. 289. 301. 320.]


In [100]:
test = [3.3,4.4,2.2,7.7]
idx = np.argsort(test)
val = np.sort(test)

print(idx,val)
    

[2 0 1 3] [2.2 3.3 4.4 7.7]
